# Chaining Qubos

This notebook contains the code used to execute a "Qubo Chain". The idea is that we can add an arbitrarily number of qubos to a chain and optimize them all together.

Instead of trying to [combine qubos into one](./CombinedQUBO.ipynb) by developping a complex qubo, we can break it up into little blocks, and evaluate the qubo chain.

In [1]:
import os 
# Path hack so notbook can open modules and files correctly:
# NOTE if you run this cell multiple times, you might need to close the notebook and re-open it
os.chdir('..') 
from services.Chain import Chain
from models.ProfitQubo import ProfitQubo
from models.SupplierQubo import SupplierQubo
from utils.data import read_inventory_optimization_data
from config import standard_mock_data
from neal import SimulatedAnnealingSampler
from dwave.system import LeapHybridDQMSampler
import numpy as np

Our chain needs to start somewhere. Start off with the inventory data.

In [2]:
data_file = standard_mock_data['small']
inventory_requirement, supplier_inventory = read_inventory_optimization_data(data_file)

When we pass data from one Qubo to the other (via the build method), we can use this function

In [3]:
def pass_data_to_next_qubo(data_file:str): 
    # Hacky function that should be generalized away
    def post_process_inventory_qubo(solution, energy):
        from utils.data import read_profit_optimization_data

        hacky_suppliers = [f'supplier{i}' for i in np.where(solution)[0]]  # looses the name, but its a hackathon.
        profit, cost = read_profit_optimization_data(data_file, hacky_suppliers)

        return dict(
            profits=profit,
            costs=cost
        )
    
    return post_process_inventory_qubo

Instantiate our qubos we want to chain

In [4]:
# Instantiate your two (or more) qubos
qubo1 = SupplierQubo(inventory_requirement, supplier_inventory)
qubo2 = ProfitQubo()  # Leave the initializer on this blank, because the data depends on the first step

# Now we can define the samplers that we want to be used on each Qubo
sampler1 = SimulatedAnnealingSampler().sample
sampler2 = LeapHybridDQMSampler().sample_dqm
samplers = [sampler1, sampler2]
# Can define kwparams for them
sampler_params = [dict(), dict()]

# Now you need to specify how to pass data from one qubo to the other
# This can definitely be improved
qubo1.define_post_process_function(pass_data_to_next_qubo(data_file))
# qubo2 does not need this because it is the last qubo in the chain

Building QUBO


Nows the fun part
### We instantiate and evaluate the chain

In [5]:
# Instantiate chain
chain = Chain()
# Add some qubos
chain.append(qubo1)
chain.append(qubo2)

# Evaluate the chain
chain.process_best(samplers, sampler_params)


Solving Qubo
Solved
  x_1 x_10 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 y_(1, 1) ... y_(9, 9) energy num_oc.
0   1    1   0   1   1   0   1   1   1   1        0 ...        0    8.0       1
['BINARY', 1 rows, 1 samples, 210 variables]
Starting next iteration with data {'profits': [14.193259698904324, 42.34503782559854, 29.08061425715131, 19.422087347158914, 39.65028317935948, 2.647142528191858, 37.02026493051393, 29.27274307589468, 7.987608230428792, 21.903774512816625, 15.163823982865704, 1.7489799922534774, 18.09133959951975, 48.225991562803394, 37.5644702135008, 34.29426458078795, 17.628685377043077, 44.79655445736397, 10.932978575753499, 3.158230974212466], 'costs': array([ 7.09662985, 21.17251891, 14.54030713,  9.71104367, 19.82514159,
        1.32357126, 18.51013247, 14.63637154,  3.99380412, 10.95188726,
        7.58191199,  0.87449   ,  9.0456698 , 24.11299578, 18.78223511,
       17.14713229,  8.81434269, 22.39827723,  5.46648929,  1.57911549])}
Building QUBO
Solving Qubo
Solved
   x0 x

### Now we can evalute the results from the qubos in the chain

1. For the first step, we minimized the number of needed suppliers to match our inventory (unfortunately, this solution is not accurate yet)

In [6]:
print('Here is the result from the first Qubo')
print(qubo1.response)
def print_suppliers(qubo):
    # Hacky function that should be generalized away
    best_solution = [qubo.response.first.sample[i] for i in qubo.x]
    return [f'supplier{i}' for i in np.where(best_solution)[0]]

print(f'\nThe "best" suppliers selected for this step: {print_suppliers(qubo1)}')

Here is the result from the first Qubo
  x_1 x_10 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 y_(1, 1) ... y_(9, 9) energy num_oc.
0   1    1   0   1   1   0   1   1   1   1        0 ...        0    8.0       1
['BINARY', 1 rows, 1 samples, 210 variables]

The "best" suppliers selected for this step: ['supplier0', 'supplier2', 'supplier3', 'supplier5', 'supplier6', 'supplier7', 'supplier8', 'supplier9']


2. For the second step, we maximized the profit. Lets check out our solution for that!

In [7]:
print('Here is the result from the second Qubo')
print(qubo2.response)

qubo1_output = qubo1.post_process
print('\nThe data passed into qubo2:')
print(qubo1_output[0])

costs_output = qubo1_output[0]['costs']
profits_output = qubo1_output[0]['profits']
print(f'\nTotal cost calculated: {np.sum([cost*count for cost, count in zip(costs_output, qubo2.solution_set)])} with budget {qubo2.budget}')
print(f'Total profit calculated: {np.sum([profit*count for profit, count in zip(profits_output, qubo2.solution_set)])} with budget {qubo2.budget}')

print('\nSummary\n')
print('Step 1: Supplier Inventory Optimization')
print(f'The "best" suppliers selected for this step: {print_suppliers(qubo1)}')

print('\nStep 2: Profit Optimization')
print(f'Total profit calculated: {np.sum([profit*count for profit, count in zip(profits_output, qubo2.solution_set)])} with budget {qubo2.budget}')

Here is the result from the second Qubo
   x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 x14 ... y6      energy num_oc.
11  0  0  0  0  0  2  0  0  0  0   0   5   0   0   0 ...  1 -184.026713       1
4   0  0  0  0  0  0  0  0  0  0   0   1   0   0   0 ...  1 -152.071184       1
9   0  0  0  0  0  1  0  0  0  0   0   0   0   0   0 ...  1  -134.44474       1
2   0  0  0  0  0  3  0  0  0  0   0   4   0   0   0 ...  1 -119.356272       1
10  0  0  0  0  0  1  0  0  0  0   0   0   0   0   0 ...  1 -119.213284       1
1   0  0  0  0  0  1  0  0  0  0   0   0   0   0   0 ...  0  -84.033014       1
3   0  0  0  0  0  1  0  0  0  0   0   1   0   0   0 ...  0  -62.103338       1
6   0  0  0  0  0  7  0  0  0  0   0   0   0   0   0 ...  0  -62.096502       1
7   0  0  0  0  0 10  0  0  0  0   0   0   0   0   0 ...  0  -61.985658       1
5   0  0  0  0  0  6  0  0  0  0   0   6   0   0   0 ...  0  -53.805253       1
8   0  0  0  0  0  8  0  0  0  0   0   8   0   0   0 ...  0  -46.076892       1


Lets just put all this logic in one function so we can re-use it

In [8]:
def run_experiment(filename): 
    inventory_requirement, supplier_inventory = read_inventory_optimization_data(filename)
    # Instantiate your two (or more) qubos
    qubo1 = SupplierQubo(inventory_requirement, supplier_inventory)
    qubo2 = ProfitQubo()  # Leave the initializer on this blank, because the data depends on the first step

    # Now we can define the samplers that we want to be used on each Qubo
    sampler1 = SimulatedAnnealingSampler().sample
    sampler2 = LeapHybridDQMSampler().sample_dqm
    samplers = [sampler1, sampler2]
    # Can define kwparams for them
    sampler_params = [dict(), dict()]

    # Now you need to specify how to pass data from one qubo to the other
    # This can definitely be improved
    qubo1.define_post_process_function(pass_data_to_next_qubo(filename))
    # qubo2 does not need this because it is the last qubo in the chain

    # Instantiate chain
    chain = Chain()
    # Add some qubos
    chain.append(qubo1)
    chain.append(qubo2)

    # Evaluate the chain
    chain.process_best(samplers, sampler_params)

    print('Here is the result from the first Qubo')
    print(qubo1.response)
    def print_suppliers(qubo):
        # Hacky function that should be generalized away
        best_solution = [qubo.response.first.sample[i] for i in qubo.x]
        return [f'supplier{i}' for i in np.where(best_solution)[0]]

    print(f'\nThe "best" suppliers selected for this step: {print_suppliers(qubo1)}')

    print('Here is the result from the second Qubo')
    print(qubo2.response)

    qubo1_output = qubo1.post_process
    # print('\nThe data passed into qubo2:')
    # print(qubo1_output[0])

    costs_output = qubo1_output[0]['costs']
    profits_output = qubo1_output[0]['profits']
    print(f'\nTotal cost calculated: {np.sum([cost*count for cost, count in zip(costs_output, qubo2.solution_set)])} with budget {qubo2.budget}')
    print(f'Total profit calculated: {np.sum([profit*count for profit, count in zip(profits_output, qubo2.solution_set)])} with budget {qubo2.budget}')

    print('\nSummary\n')
    print('Step 1: Supplier Inventory Optimization')
    print(f'The "best" suppliers selected for this step: {print_suppliers(qubo1)}')

    print('\nStep 2: Profit Optimization')
    print(f'Total profit calculated: {np.sum([profit*count for profit, count in zip(profits_output, qubo2.solution_set)])} with budget {qubo2.budget}')

We can run this experiment on all our data sets

In [9]:
run_experiment(standard_mock_data['small'])

Building QUBO
Solving Qubo
Solved
  x_1 x_10 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 y_(1, 1) ... y_(9, 9) energy num_oc.
0   0    1   0   1   1   1   1   1   1   0        0 ...        0   17.0       1
['BINARY', 1 rows, 1 samples, 210 variables]
Starting next iteration with data {'profits': [14.193259698904324, 42.34503782559854, 29.08061425715131, 19.422087347158914, 39.65028317935948, 2.647142528191858, 37.02026493051393, 29.27274307589468, 7.987608230428792, 21.903774512816625, 15.163823982865704, 1.7489799922534774, 18.09133959951975, 42.77435773396476, 37.5644702135008, 34.29426458078795, 19.35323068566685, 44.79655445736397, 10.932978575753499, 3.158230974212466], 'costs': array([ 7.09662985, 21.17251891, 14.54030713,  9.71104367, 19.82514159,
        1.32357126, 18.51013247, 14.63637154,  3.99380412, 10.95188726,
        7.58191199,  0.87449   ,  9.0456698 , 21.38717887, 18.78223511,
       17.14713229,  9.67661534, 22.39827723,  5.46648929,  1.57911549])}
Building QUBO
Solving Qubo
So

In [10]:
run_experiment(standard_mock_data['medium'])

Building QUBO
Solving Qubo
Solved
  x_1 x_10 x_11 x_12 x_13 x_14 x_15 x_16 x_17 x_18 ... y_(99, 9) energy num_oc.
0   1    1    1    1    1    1    1    0    1    1 ...         0  891.0       1
['BINARY', 1 rows, 1 samples, 4040 variables]
Starting next iteration with data {'profits': [38.19056030528154, 6.2221736291464795, 36.95941005668472, 7.212357138553866, 37.09734191729193, 36.297925567920984, 37.803690863596714, 29.465655226937848, 23.286939891154496, 5.299257882533357, 42.46298328633512, 5.028537687708292, 31.404606574002635, 0.9720282256865127, 14.054803795147318, 7.930503449040431, 42.207172145007874, 40.87149988742412, 31.11318075917573, 40.983148848748115, 42.14704075932852, 25.58621797141423, 6.838982787675635, 4.3243957130040105, 43.59372003788174, 26.60761142564833, 10.026364040437777, 7.17870622388254, 29.255451878316684, 7.185719143317469, 17.806749414865216, 13.98710135014695, 24.897917493054116, 34.556767224624934, 16.326955895805916, 2.610526998025086, 28.6084140637

In [11]:
run_experiment(standard_mock_data['large'])

Building QUBO
Solving Qubo
